### Ecole des Mines de Paris
### Problemes Inverses - S1923

# Projet nº5 - Tomographie à rayons parallèles



<blockquote> Ce projet consiste à étudier le problème de retrouver la forme d’un objet à partir d’une technique de tomographie à rayons parallèles. Ceci est fait en émettant des rayons électromagnétiques parallèlement d’un côté de l’objet et ensuite en les mesurant de l’autre côté, comme montre la Figure 1. L’intensité mesuré des rayons étant dépendante des propriétés du milieu que ceux-ci ont traversé (représentées par la fonction f(x; y) dans la Figure 1), on peut calculer la valeur de ces propriétés à chaque point (x; y) de l’espace. Ce processus est répété pour différents angles θ de façon à balayer tout l’objet par différentes perspectives, étant ainsi possible de reconstruire
son allure. </blockquote>


 <img src="Rayons.PNG" width ="400" height=400 > 
$\hspace{7.5cm}$ Figure 1

Le modèle est donné par : <br>
$$ b = ln(\frac{I}{I_0}) = -\int_{\gamma} \mu(x,y)dl $$

o`u I est l’intensit´e mesur´ee du rayon, I0 est son intensit´e ´emise et le chemin γi correspond `a
un rayon fix´e. Calculer b sachant f est le probl`eme directe, lequel est lin´eaire. Nous cherchons
`a r´esoudre le probl`eme inverse : `a partir de quelques mesures de b, retrouver f.
Pour faire cette r´esolution num´eriquement, on divise le milieu en N2 cellules carr´ees, comme
montre la Figure 2. On choisi aussi une quantit´e finie nθ de valeurs pour l’angle θ et un nombre
p de rayons ´emis. Dans ce cas-ci, l’´equation (1) devient
b = Af (2)
o`u A est la matrice qui correspond `a la discr´etisation de l’int´egrale de l’´equation (1). Le vecteur
b a nθ × p entr´ees, et le vecteur f a N2.
2

In [153]:
import numpy as np
from numpy import matlib as mb
import math

def makephantom(N,example):

# %MAKEPHANTOM creates the modified Shepp-Logan phantom
# %   X = myphantom(N)
# % 
# % This function create the modifed Shepp-Logan phantom with the
# % discretization N x N, and returns it as a vector.
# %
# % Input:
# %   N    Scalar denoting the nubmer of discretization intervals in each
# %        dimesion, such that the phantom head consists of N^2 cells.
# % 
# % Output:
# %   X    The ellipse phantom  reshaped as a vector

# %         A    a     b    x0    y0    phi
# %        ---------------------------------


    if(example == 1):
        e = np.array([  1  , .29 , .52  ,  0   ,  0   ,  0   ])
    if(example == 2):
        e = np.array([  1  , .52  , .29  ,  0  ,   0  ,   0   ])
    if(example == 3):
        e = np.array([  1  , .52  , .29  ,  0   ,  0   ,  45  ])
    

    xn = np.linspace(-1,1,N)
    Xn = mb.repmat(xn,N,1);
    Yn = np.rot90(Xn);
    X = np.zeros(N*N);

    # For each ellipse to be added     
    # for i in range(e.shape[0]):
    for i in range(1):
        a2 = e[1]**2
        b2 = e[2]**2
        x0 = e[3]
        y0 = e[4]
        phi = e[5]*math.pi/180
        A = e[0]

        x = Xn-x0;
        y = Yn-y0;
        
        v = ( ((x*math.cos(phi) + y*math.sin(phi))**2)/a2 + 
            ((y*math.cos(phi) - x*math.sin(phi)))**2/b2 )

        
        
    # Add the amplitude of the ellipse
        v = np.ravel(v.T)
#         print(v<=1)
#         print(X)
        X[v<=1] = X[v<=1]+A


# Return as vector and ensure nonnegative elements.
    X = np.ravel(X)
    X[X<0] = 0
    
    return X



In [ ]:
function [A theta p d] = paralleltomo(N,theta,p,d,isDisp)
%PARALLELTOMO Creates a 2D tomography test problem using parallel beams
%
%   [A theta p d] = paralleltomo(N)
%   [A theta p d] = paralleltomo(N,theta)
%   [A theta p d] = paralleltomo(N,theta,p)
%   [A theta p d] = paralleltomo(N,theta,p,d)
%   [A theta p d] = paralleltomo(N,theta,p,d,isDisp)
%
% This function creates a 2D tomography test problem with an N-times-N
% domain, using p parallel rays for each angle in the vector theta.
%
% Input: 
%   N           Scalar denoting the number of discretization intervals in 
%               each dimesion, such that the domain consists of N^2 cells.
%   theta       Vector containing the angles in degrees. Default: theta = 
%               0:1:179.
%   p           Number of parallel rays for each angle. Default: p =
%               round(sqrt(2)*N).
%   d           Scalar denoting the distance from the first ray to the last.
%               Default: d = sqrt(2)*N.
%   isDisp      If isDisp is non-zero it specifies the time in seconds 
%               to pause in the display of the rays. If zero (the default), 
%               no display is shown.
%
% Output:
%   A           Coefficient matrix with N^2 columns and nA*p rows, 
%               where nA is the number of angles, i.e., length(theta).
%   theta       Vector containing the used angles in degrees.
%   p           The number of used rays for each angle.
%   d           The distance between the first and the last ray.
% 
% See also: fanbeamtomo, seismictomo.

% Jakob Heide Jï¿½rgensen, Maria Saxild-Hansen and Per Christian Hansen,
% June 21, 2011, DTU Informatics.

% Reference: A. C. Kak and M. Slaney, Principles of Computerized 
% Tomographic Imaging, SIAM, Philadelphia, 2001.

% Default illustration:
if nargin < 5 || isempty(isDisp)
    isDisp = 0;
end

% Default value of d.
if nargin < 4 || isempty(d)
    d = sqrt(2)*N;
end

% Default value of the number of rays.
if nargin < 3 || isempty(p)
    p = round(sqrt(2)*N);
end

% Default value of the angles theta.
if nargin < 2 || isempty(theta)
    theta = 0:179;
end

% Define the number of angles.
nA = length(theta);

% The starting values both the x and the y coordinates. 
x0 = linspace(-d/2,d/2,p)';
y0 = zeros(p,1);

% The intersection lines.
x = (-N/2:N/2)';
y = x;

% Initialize vectors that contains the row numbers, the column numbers and
% the values for creating the matrix A effiecently.
rows = zeros(2*N*nA*p,1);
cols = rows;
vals = rows;
idxend = 0;

% Prepare for illustration
if isDisp
    AA = rand(N);
    figure
end

% Loop over the chosen angles.
for i = 1:nA    
    
    % Illustration of the domain
    if isDisp
        clf
        pause(isDisp)
        imagesc((-N/2+.5):(N/2-0.5),(-N/2+.5):(N/2-0.5),AA), colormap gray,
        hold on
        axis xy
        axis equal
        axis([-N/2-1 N/2+1 -N/2-1 N/2+1])        
    end
    
    % All the starting points for the current angle.
    x0theta = cosd(theta(i))*x0-sind(theta(i))*y0;
    y0theta = sind(theta(i))*x0+cosd(theta(i))*y0;
    
    % The direction vector for all the rays corresponding to the current 
    % angle.
    a = -sind(theta(i));
    b = cosd(theta(i));
    
    % Loop over the rays.
    for j = 1:p
        
        % Use the parametrisation of line to get the y-coordinates of
        % intersections with x = k, i.e. x constant.
        tx = (x - x0theta(j))/a;
        yx = b*tx + y0theta(j);
        
        % Use the parametrisation of line to get the x-coordinates of
        % intersections with y = k, i.e. y constant.
        ty = (y - y0theta(j))/b;
        xy = a*ty + x0theta(j);

        % Illustration of the rays
        if isDisp           
            
            plot(x,yx,'-','color',[220 0 0]/255,'linewidth',1.5)
            plot(xy,y,'-','color',[220 0 0]/255,'linewidth',1.5)
                     
            set(gca,'Xticklabel',{})
            set(gca,'Yticklabel',{})
            pause(isDisp)
        end
        
        % Collect the intersection times and coordinates. 
        t = [tx; ty];
        xxy = [x; xy];
        yxy = [yx; y];
        
        % Sort the coordinates according to intersection time.
        [t I] = sort(t);
        xxy = xxy(I);
        yxy = yxy(I);        
        
        % Skip the points outside the box.
        I = (xxy >= -N/2 & xxy <= N/2 & yxy >= -N/2 & yxy <= N/2);
        xxy = xxy(I);
        yxy = yxy(I);
        
        % Skip double points.
        I = (abs(diff(xxy)) <= 1e-10 & abs(diff(yxy)) <= 1e-10);
        xxy(I) = [];
        yxy(I) = [];
        
        % Calculate the length within cell and determines the number of
        % cells which is hit.
        d = sqrt(diff(xxy).^2 + diff(yxy).^2);
        numvals = numel(d);
        
        % Store the values inside the box.
        if numvals > 0
            
            % If the ray is on the boundary of the box in the top or to the
            % right the ray does not by definition lie with in a valid cell.
            if ~((b == 0 && abs(y0theta(j) - N/2) < 1e-15) || ...
                 (a == 0 && abs(x0theta(j) - N/2) < 1e-15)       )
                
                % Calculates the midpoints of the line within the cells.
                xm = 0.5*(xxy(1:end-1)+xxy(2:end)) + N/2;
                ym = 0.5*(yxy(1:end-1)+yxy(2:end)) + N/2;
                
                % Translate the midpoint coordinates to index.
                col = floor(xm)*N + (N - floor(ym));
                
                % Create the indices to store the values to vector for
                % later creation of A matrix.
                idxstart = idxend + 1;
                idxend = idxstart + numvals - 1;
                idx = idxstart:idxend;
                
                % Store row numbers, column numbers and values. 
                rows(idx) = (i-1)*p + j;
                cols(idx) = col;
                vals(idx) = d;   
                
            end
        end
        
    end
end

% Truncate excess zeros.
rows = rows(1:idxend);
cols = cols(1:idxend);
vals = vals(1:idxend);

% Create sparse matrix A from the stored values.
A = sparse(rows,cols,vals,p*nA,N^2);


In [ ]:
####  PYTHON MODULES
from __future__ import division , print_function
import numpy as np
from scipy.sparse import lil_matrix
import myPrint as pp
import myImageDisplay as dis
import myImageIO as io
import sys



####  MY FORMAT VARIABLE
myfloat = np.float32




####  MY CONSTANTS
eps = 1e-10
infp = 1e20
infn = -infp    



###########################################################
###########################################################
####                                                   ####
####          MATRIX REPRESENTATION FOR THE            ####
####   FORWARD PROJECTOR FOR PARALLEL BEAM GEOMETRY    ####
####                                                   ####
####   Python translation of paralleltomo.m inside     ####
####        the AIRtools package of C. Hansen          ####
####                                                   ####
###########################################################
###########################################################

####  Inputs:
####  angles  --->  array of projection angles ( given in degrees )
####  N       --->  total number of pixel of the image ( sqrt(N) x sqrt(N) ) 
####  p       --->  number of pixels for each projection; it is such that: p >= N
####
####  Output:
####  A       ---> matrix representation of the forward projector

def paralleltomo( angles , N , p , d ):
    ##  Convert angles to radiants
    theta = angles * np.pi / 180.0

    ##  Allocate memory for sparse matrix storing the forward projector
    M = len( angles )
    dim1 = M * N; dim2 = N * N
    A = lil_matrix( ( dim1 , dim2 ) , dtype=myfloat )
    #A = np.zeros( ( dim1 , dim2 ) , dtype=myfloat )


    ##  Starting values for both the x and y coordinates
    x0 = np.linspace( -0.5*d , 0.5*d , p )
    y0 = np.zeros( p , dtype=myfloat )
    
    #print('\nx0:', x0)


    ##  Intersection lines
    x = np.arange( -0.5*N , 0.5*N + 1 )
    y = x.copy()
    #print('x: ', x)


    ##  Initialize vectors that will be used inside the main loop
    x0theta = x0.copy(); y0theta = y0.copy()
    tx = x.copy(); ty = x.copy(); xy = x.copy(); yx = x.copy()
    t = np.zeros( 2 * len( x ) , dtype=myfloat )


    ##  Creating elements of the forward projector
    ##  Loop on the angles
    for i in range( M ): # M
        print('\ni = ', i)

        ##  Get all the starting points for the current angle
        s = np.sin( theta[i] ) ; c = np.cos( theta[i] )

        #print('s = ', s)
        #print('c = ', c)
        #print('x0 = ', x0)
        #print('y0 = ', y0)

        x0theta[:] = c * x0 - s * y0
        y0theta[:] = s * x0 + c * y0

        #print( '\nx0theta = ', x0theta )
        #print( '\ny0theta = ', y0theta )   


        ##  Loop on the rays
        for j in range( p ):  # p
            #print( '\nj = ' , j )

            ##  Use the parametrisation of line to get the y-coord.
            ##  of intersections with x = k, i.e. x constant
            if np.abs( s ) > eps:
                tx[:] =  - ( x - x0theta[j] ) / s
            else:
                tx[ (-( x - x0theta[j]) >= 0 ) ] = infp
                tx[ (-( x - x0theta[j] ) < 0 ) ] = infn
            yx[:] = c * tx + y0theta[j] 
                

            #print('tx = ', tx)
            #print('yx = ', yx)


            ##  Use the parametrisation of line to get the x-coord.
            ##  of intersections with y = k, i.e. y constant
            if np.abs( c ) > eps:             
                ty[:] = ( y - y0theta[j] ) / c
            else:
                ty[ ( ( y - y0theta[j] ) >= 0 ) ] = infp
                ty[ ( ( y - y0theta[j] ) < 0 ) ] = infn    
            xy[:] =  -s * ty + x0theta[j]    

            #print('ty = ', ty)
            #print('xy = ', xy)


            ##  Collect the intersections and coordinates
            t[:] = np.concatenate( ( tx , ty ) , axis=1 )
            xxy = np.concatenate( ( x , xy ) , axis=1 )
            yxy = np.concatenate( ( yx , y ) , axis=1 )

            #print('t = ', t)
            #print('xxy = ', xxy)
            #print('yxy = ', yxy)


            ##  Sort the acoordinates according to intersection time
            I = np.argsort( t )  
            t[:] = np.sort( t )
            xxy[:] = xxy[I]
            yxy[:] = yxy[I] 

            #print('t sort = ', t)
            #print('I = ', I)
    

            ##  Skip the points outside the box and double points
            #print( np.argwhere( ( xxy >= -N/2 ) & ( xxy <= N/2 ) & \
            #                   ( yxy >= -N/2 ) & ( yxy <= N/2 ) ) )
            #print( np.argwhere( np.abs( np.diff( xxy ) ) > eps ) )
            #print( np.argwhere( np.abs( np.diff( yxy ) ) > eps ) ) 
            I = np.argwhere( ( xxy >= -N/2 ) & ( xxy <= N/2 ) & \
                               ( yxy >= -N/2 ) & ( yxy <= N/2 ) )
                               #( np.abs( np.diff( xxy ) ) > eps ) & \
                               #( np.abs( np.diff( yxy ) ) > eps ) )
            xxy = xxy[I]
            yxy = yxy[I]

            #print('I = ', I)
            #print('xxy = ', xxy)
            #print('yxy = ', yxy)
            
            
            ##  Skip double points
            I = np.argwhere( ( np.abs( xxy[1:] - xxy[:-1] ) < eps ) & \
                             ( np.abs( yxy[1:] - yxy[:-1] ) < eps ) )
            if len( I ) != 0:
                np.delete( xxy , I )
                np.delete( yxy , I )

            #print('I 2nd = ', I)
            #print('xxy 2nd = ', xxy)
            #print('yxy 2nd = ', yxy) 


            ##  Calculate the length within cell and determines the
            ##  number of cells which gets hit
            d = np.sqrt( ( xxy[1:] - xxy[:-1] )**2 + ( yxy[1:] - yxy[:-1] )**2 )
            n_hit = len( d )

            #print('d = ', d)
            #print('n_hit = ', n_hit)


            ##  Store the values inside the box
            if n_hit > 0:
                ##  Neglect rays on the boundaries of the box
                cond1 = ( s == 0.0 ) and ( np.abs( x0theta[j] - N/2 ) < eps )
                cond2 = ( c == 0.0 ) and ( np.abs( y0theta[j] - N/2 ) < eps )

                #print( cond1 )
                #print( cond2 )
                
                if( not( cond1 or cond2 ) ):

                    ##  Calculate the midpoints of the line within the cells
                    xm = 0.5 * ( xxy[:-1] + xxy[1:] )  + N/2
                    ym = 0.5 * ( yxy[:-1] + yxy[1:] )  + N/2

                    #print('xm = ', xm)
                    #print('ym = ', ym)


                    ##  Translate the midpoint coordinates to index
                    cols = np.floor( xm ) * N + ( N - 1 - np.floor( ym ) )
                    cols = cols.astype( int )
                    cols = cols.reshape(-1)

                    #print('cols shape = ', cols.shape)
                    #print('cols = ', cols)


                    ##  Get rows coordinates
                    row = i * p + j
                    #print('row = ', row)

                    #print( A. shape )


                    ##  Fill the matrix
                    A[row, cols] = d
                     
    return A

In [154]:
e = np.array([  1  , .29 , .52  ,  0   ,  0   ,  0   ])
phi = e[4]*math.pi/180

X = makephantom(5,1)
print(X)

[False False False False False False False False False False False  True
  True  True False False False False False False False False False False
 False]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
